In [1]:
import plotly.express as px
from plotly.subplots import make_subplots
from weather_data import WeatherData, ModelBasedOptions, HourlyData
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
latitude = 50.732817
longitude = 16.648050

start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 1, 31)

options = ModelBasedOptions(
    hourly=[
        HourlyData.Temperature_2m,
        HourlyData.RelativeHumidity_2m,
        HourlyData.WindDirection_10m,
        HourlyData.WindSpeed_10m,
        HourlyData.Precipitation_rain_showers_snow,
    ]
)
meta_data_model, daily_model, hourly_model = WeatherData.getModelBasedData(
    latitude, longitude, start_date, end_date, options
)

meta_data_station, daily_station, hourly_station = WeatherData.getStationData(
    latitude, longitude, start_date, end_date, require_daily=False, require_hourly=True, skip_stations=["12150"]
)


In [3]:
fig = px.scatter_mapbox(pd.DataFrame({"lat": [latitude], "lng": [longitude]}), lat="lat", lon="lng", zoom=5, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()


In [15]:
df = pd.read_csv("../../data/pollution/raw/2022/pm10/2022_16.csv")
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S")

# JANUAR
januar = df[
    (df["date"] >= pd.Timestamp(year=2022, month=1, day=1))
    & (df["date"] <= pd.Timestamp(year=2022, month=1, day=31, hour=23, minute=59))
]
result_januar = januar.groupby(januar.index // 24).agg({"value": "mean"}).round(5)
result_januar["date"] = index = pd.date_range("2022-01-01", periods=31)
result_januar = result_januar.set_index("date")

# add missing values to Januar
januar = januar.__deepcopy__()
januar.loc[194.5] = ["2022-01-09 03:00:00", 0.00]
januar.loc[194.7] = ["2022-01-09 04:00:00", 0.00]

januar = januar.sort_index().reset_index(drop=True)


In [7]:
pm10 = januar["value"]

temp = hourly_model["temperature_2m"]
humidity = hourly_model["relativehumidity_2m"]
winddirection = hourly_model["winddirection_10m"]
windspeed = hourly_model["windspeed_10m"]
precipitation = hourly_model["precipitation"]

print(pm10.size, temp.size, humidity.size, winddirection.size, windspeed.size, precipitation.size)


744 744 744 744 744 744


In [16]:
correlation_df = pd.DataFrame(
    {
        "temp": temp.to_numpy(),
        "humidity": humidity.to_numpy(),
        "winddirection": winddirection.to_numpy(),
        "windspeed": windspeed.to_numpy(),
        "precipitation": precipitation.to_numpy(),
        "pm10": pm10.to_numpy(),
    }
)


In [7]:
# sns.heatmap(correlation_df.corr(), vmin=-1, vmax=1, annot=True, cmap="rocket_r")


### Korrelation über einen Monat (01.2022)


In [17]:
fig = px.imshow(correlation_df.corr().round(2), text_auto=True, color_continuous_scale="RdBu", zmin=-1, zmax=1)


fig.update_layout({"height": 480, "width": 600}, title_text="Correlation, station 814, Jan 22", title_x=0.5)


In [9]:
px.imshow(correlation_df.corr("spearman").round(2), text_auto=True, color_continuous_scale="RdBu")


### Korrelation über einen Tag (01.01.2022)


In [10]:
correlation_day_df = correlation_df.loc[0:23]
px.imshow(correlation_day_df.corr().round(2), text_auto=True, color_continuous_scale="RdBu")


In [11]:
px.imshow(correlation_day_df.corr("spearman").round(2), text_auto=True, color_continuous_scale="RdBu")


### Korrelation über 14 Tage


In [12]:
correlation_month_df = correlation_df.loc[0:167]
px.imshow(correlation_month_df.corr().round(2), text_auto=True, color_continuous_scale="RdBu")


In [13]:
px.imshow(correlation_month_df.corr("spearman").round(2), text_auto=True, color_continuous_scale="RdBu")


### Korrelation in einem Zeitraum ohne Regen


In [14]:
correlation_rain_df = correlation_df.loc[225:301]
px.imshow(correlation_rain_df.corr().round(2), text_auto=True, color_continuous_scale="RdBu")


In [15]:
px.imshow(correlation_rain_df.corr("spearman").round(2), text_auto=True, color_continuous_scale="RdBu")


In [13]:
date_time = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S")
timestamp_s = date_time.map(pd.Timestamp.timestamp)

day = 24 * 60 * 60
year = (365.2425) * day

correlation_df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
correlation_df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
correlation_df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
correlation_df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))


In [17]:
df_year = pd.read_csv("../../data/pollution/raw/2022/2022_813_5349.csv")
df_year["date"] = pd.to_datetime(df_year["date"], format="%Y-%m-%d %H:%M:%S")
df_year = df_year.set_index("date")
df_year = df_year.rename(columns={"value": "pm10"})

df_year_pm25 = pd.read_csv("../../data/pollution/raw/2022/2022_813_25931.csv")
df_year_pm25["date"] = pd.to_datetime(df_year_pm25["date"], format="%Y-%m-%d %H:%M:%S")
df_year_pm25 = df_year_pm25.set_index("date")
df_year_pm25 = df_year_pm25.rename(columns={"value": "pm25"})

data_range = pd.date_range(datetime(2022, 1, 1), datetime(2022, 12, 31, 23, 59, 59), freq="1H").transpose()

df_base = pd.DataFrame(data_range, columns=["date"])
df_base = df_base.set_index("date")

df_base = df_base.join(df_year)
df_base = df_base.join(df_year_pm25)

df_base = df_base.interpolate(method="time", limit=10, limit_direction="both", limit_area="inside", inplace=False)

day = 24 * 60 * 60
year = (365.2425) * day

# timestamp_s = df_base.index.map(pd.Timestamp.timestamp)

# df_base["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
# df_base["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
# df_base["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
# df_base["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))

options = ModelBasedOptions(
    hourly=[
        HourlyData.Temperature_2m,
        HourlyData.RelativeHumidity_2m,
        HourlyData.WindDirection_10m,
        HourlyData.WindSpeed_10m,
        HourlyData.Precipitation_rain_showers_snow,
    ]
)
_, _, weather = WeatherData.getModelBasedData(50.246795, 19.019469, df_base.index[0], df_base.index[-1], options)

df_base = df_base.join(weather)
df_base = df_base.rename(
    columns={
        "temperature_2m": "temperature",
        "relativehumidity_2m": "relativehumidity",
        "winddirection_10m": "winddirection",
        "windspeed_10m": "windspeed",
    }
)

fig = px.imshow(
    df_base.corr().round(2),
    text_auto=True,
    color_continuous_scale="RdBu",
    zmin=-1,
    zmax=1,
    title="Correlation analysis 2022, station 813 (Katowice)",
    height=600,
    width=750,
)
fig.update_layout(title_x=0.5)


In [22]:
df_base


,pm10,pm25,temperature,relativehumidity,winddirection,windspeed,precipitation
2022-01-01 00:00:00,19.7521,14.82500,8.3,89,247,20.0,0.0
2022-01-01 01:00:00,31.2880,24.82080,8.1,90,253,21.4,0.0
2022-01-01 02:00:00,26.0966,20.78290,8.3,89,190,13.9,0.1
2022-01-01 03:00:00,15.3508,12.37000,8.3,90,182,12.6,0.4
2022-01-01 04:00:00,11.9743,9.91801,8.3,92,178,13.7,0.1
...,...,...,...,...,...,...,...
2022-12-31 19:00:00,15.3923,12.36910,9.5,75,225,26.0,0.0
2022-12-31 20:00:00,15.2635,11.77000,9.5,75,226,26.0,0.0
2022-12-31 21:00:00,14.9436,11.27920,10.3,72,225,24.9,0.0
2022-12-31 22:00:00,14.7588,12.00560,10.3,71,224,23.7,0.0


In [18]:
fig = px.imshow(
    df_base.corr("spearman").round(2),
    text_auto=True,
    color_continuous_scale="RdBu",
    zmin=-1,
    zmax=1,
    title="Correlation analysis 2022, station 813 (Katowice)",
    height=600,
    width=750,
)
fig.update_layout(title_x=0.5)


In [19]:
df_extreme = df_base[df_base["pm10"] >= 50]

fig = px.imshow(
    df_extreme.corr().round(2),
    text_auto=True,
    color_continuous_scale="RdBu",
    zmin=-1,
    zmax=1,
    title="Correlation analysis 2022, station 813 (Katowice); PM10 values >120ug/m^3",
    height=600,
    width=750,
)
fig.update_layout(title_x=0.5)


In [20]:
fig = px.imshow(
    df_extreme.corr("spearman").round(2),
    text_auto=True,
    color_continuous_scale="RdBu",
    zmin=-1,
    zmax=1,
    title="Correlation analysis 2022, station 813 (Katowice); PM10 values >120ug/m^3",
    height=600,
    width=750,
)
fig.update_layout(title_x=0.5)


> 120 : 13
> 100 : 80
> 50 : 1051


In [21]:
df_extreme.shape


(1051, 7)